**Table of contents**<a id='toc0_'></a>    
- [Estimateurs : comparaison et recherche de fuite](#toc1_)    
- [Tableau comparatif](#toc2_)    
- [Méthode pas à pas](#toc3_)    
- [Enregistrement des traitements](#toc4_)    
- [Scores (r2) selon les traitements](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=1
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import logging
import pickle
import time
from functools import partial
import warnings

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# render in GitHub & NBViewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# prevent warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# logging configuration (see all outputs, even DEBUG or INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
DATASETS_PATH = "./"
dataset_name = "oc_p3_2016_Building_Energy_Benchmarking.csv"
data_raw = pd.read_csv(DATASETS_PATH+dataset_name)

In [3]:
def nan_warn(df, nan_col="nan_pct", tags_col="tags", thresh=0.4):
    """
    Warns if NaNs outpass a defined threshold,
    warning showed as a tag in the dataframe on a defined column.

    Inputs:
    • df: dataframe
    • nan_col: dataframe column (string, default = "nan")
    • tags_col: dataframe column (string, default = "tags")
    • thresh: threshold for NaNs warning (float, default = 0.4)

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    mask = df_[nan_col] / 100 >= thresh
    df_.loc[mask, tags_col] = df_.loc[mask, tags_col] + "🚫"

    return df_

def type_tag(df, uni_col="unique", type_col="type", count_col="count",
    tags_col="tags"):
    """
    Defines a type tag of a dataframe feature,
    depending on unique values, count and dtype,
    and writes it in a tag column.

    Inputs:
    • df: dataframe
    • uni_col: dataframe column (string, default = "unique")
    • type_col: dataframe column (string, default = "type")
    • count_col: dataframe column (string, default = "type")
    • tags_col: dataframe column (string, default = "tags")

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    total_count = max(df_[count_col])

    # const warn
    const_mask = df_[uni_col] == 1
    df_.loc[const_mask, tags_col] = df_.loc[const_mask, tags_col] + "🔒"

    # unique warn
    uniq_mask = df_[uni_col] == total_count
    df_.loc[uniq_mask, tags_col] = df_.loc[uniq_mask, tags_col] + "💎"

    # bool = categorical feat
    is_bool_mask = df_[type_col] == "bool"
    df_.loc[is_bool_mask, tags_col] = df_.loc[is_bool_mask, tags_col] + "📦"
    
    # object categorical feat
    type_mask = df_[type_col] == "object"
    # define limit
    categ_limit = int(max(2, min(60, total_count / 1.2)))
    # filter
    categ_mask = df_[uni_col].between(2, categ_limit)
    df_.loc[(categ_mask & type_mask), tags_col] = df_.loc[
        (categ_mask & type_mask), tags_col] + "📦"

    return df_

def describe_df(df, nan_thresh=0.4):
    """
    Dataframe describer, include little more information than .describe()

    Inputs:
    • df: dataframe to be analysed
    • nan_thresh: threshold for NaNs warning (float, default = 0.4)

    Output: dataframe of data description

    Requirements: pandas, numpy
    """

    df_ = df.describe(include="all").T
    df_.sort_index(inplace=True)
    df_["unique"] = df.nunique()
    df_["type"] = df.dtypes
    df_["nan"] = df.isna().sum()
    df_["nan_pct"] = np.round(df.isna().mean()*100, 2)
    
    # tags column
    df_.insert(0, "tags", "")
    # nan warning tag
    df_ = nan_warn(df_, thresh=nan_thresh)
    # type check + const warn tag
    df_ = type_tag(df_,
        uni_col="unique",
        type_col="type",
        count_col="count",
        tags_col="tags",
    )
    
    df_ = df_.fillna("-")

    return df_

def impact_classif(value, thresh=30):
    """
    Returns an impact classification depending on a value
    and a threshold.

    Positional arguments: 
    -------------------------------------
    value: float or int: between 0 and 100

    Optional arguments: 
    -------------------------------------
    thresh: float or int: threshold to adjust the function, default=30

    Output: string, warning intensity (int)
    """

    if value == 0:
        return "⌀", False
    elif 0 < value < (thresh / 6):
        return "--", False
    elif (thresh / 6) <= value < (thresh / 3):
        return "-", False
    elif (thresh / 3) <= value < (thresh * 2 / 3):
        return "+", False
    elif (thresh * 2 / 3) <= value < thresh:
        return "++", False
    elif thresh <= value < (thresh + (thresh / 3)):
        return "⚠️", 1
    elif (thresh + (thresh / 3)) <= value < (thresh + (2 * thresh / 3)):
        return "⚠️⚠️", 2
    elif (thresh + (2 * thresh / 3)) <= value < 75:
        return "⚠️⚠️⚠️", 3
    elif value >= 75:
        return "☠️", 4
    else:
        return "❓", False

def impact(df_before, df_after, monitored=None):
    """
    Returns an impact dataframe from an original and
    a second dataframe.
    Impact is calculated on the columns and population,
    plus on some optional arguments.

    Positional arguments: 
    -------------------------------------
    df_before: dataframe: original dataframe (starting point, before action)
    df_before: dataframe: original dataframe (starting point, after action)

    Optional arguments: 
    -------------------------------------
    monitored: list of strings: Columns to check.
               ⚠️ Columns must be present in both dataframes.
               Default = None

    Output: dataframe in logging.info()

    Required modules: pandas, numpy, logging
    """

    pop_bef = df_before.shape[0]
    cols_bef = df_before.shape[1]
    
    pop_aft = df_after.shape[0]
    cols_aft = df_after.shape[1]

    diff_pop = pop_bef - pop_aft
    diff_cols = cols_bef - cols_aft

    prct_pop_num = np.round(diff_pop / pop_bef * 100, 2)
    prct_cols_num = np.round(diff_cols / cols_bef * 100, 2)
    prct_pop = f"{prct_pop_num}%"
    prct_cols = f"{prct_cols_num}%"

    imp_cols = impact_classif(prct_cols_num)
    imp_pop = impact_classif(prct_pop_num)

    # list of potential warnings
    warn_logs = [imp_cols[1], imp_pop[1]]

    _df_ = pd.DataFrame([
        [cols_bef, pop_bef],
        [cols_aft, pop_aft],
        [diff_cols, diff_pop],
        [prct_cols, prct_pop],
        [imp_cols[0], imp_pop[0]]],
        columns=["Columns", "Population"],
        index=["Before", "After", "Difference", "Prct", "IMPACT"]
    )
    

    if monitored:
        for m in monitored:
            # get output from command
            before_ = df_before[m].count()
            after_ = df_after[m].count()
            prct_ = np.round((before_ - after_) / before_ * 100, 2)
            imp = impact_classif(prct_)
            # add in DF
            _df_[m] = [before_,
                    after_,
                    before_ - after_,
                    f"{prct_}%",
                    imp[0]
            ]
            # add potential warning
            warn_logs.append(imp[1])

    if (1 in warn_logs or 2 in warn_logs or 3 in warn_logs):
        return logging.warning(display(_df_))
    elif 4 in warn_logs:
        return logging.critical(display(_df_))
    else:
        return logging.info(display(_df_))

def data_cleaner(df, min_nr_pct=0.0001, verbose=False):
    """

    Data cleaning, grouping explicitly all previously used cleaning methods.

    Input: dataframe
    Output: cleansed dataframe

    Requirements: numpy, pandas, logging
    """

    # DROP DUPLICATES
    # *************************************************************************
    df_ = df.drop_duplicates()

    # KEEP COMPLIANT DATA ONLY
    # *************************************************************************
    df_ = df_.loc[df_["ComplianceStatus"] == "Compliant"]

    # CLEAN NEIGHBORHOOD
    # *************************************************************************
    # case harmonization
    df_["Neighborhood"] = df_["Neighborhood"].str.upper()
    # duplicate deletion
    df_.loc[df_["Neighborhood"] == "DELRIDGE NEIGHBORHOODS",
        "Neighborhood"] = "DELRIDGE"
    
    # DELETE USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "Address",
        "City",
        "Comments",
        "ComplianceStatus",
        "CouncilDistrictCode",
        "DataYear",
        "DefaultData",
        # 'Electricity(kBtu)',
        "Electricity(kWh)",
        # "ENERGYSTARScore",
        # 'GHGEmissionsIntensity',
        "Latitude",
        "ListOfAllPropertyUseTypes",
        "Longitude",
        # 'NaturalGas(kBtu)',
        "NaturalGas(therms)",
        # "Neighborhood",
        # "NumberofBuildings",
        # "NumberofFloors",
        "OSEBuildingID",
        "Outlier",
        # 'PropertyGFABuilding(s)',
        # 'PropertyGFAParking',
        # 'PropertyGFATotal',
        "PropertyName",
        "SiteEnergyUse(kBtu)",
        "SiteEnergyUseWN(kBtu)",
        "SiteEUI(kBtu/sf)",
        # 'SiteEUIWN(kBtu/sf)',
        "SourceEUI(kBtu/sf)",
        "SourceEUIWN(kBtu/sf)",
        "State",
        # 'SteamUse(kBtu)',
        "TaxParcelIdentificationNumber",
        "TotalGHGEmissions",
        # 'YearBuilt',
        "YearsENERGYSTARCertified",
        "ZipCode",
        ], axis=1, inplace=True)
    
    # BUILDING RATIO
    # *************************************************************************
    df_["BuildingRatio"] = df_["PropertyGFABuilding(s)"]\
        / df_["PropertyGFATotal"]
    
    # PARKING RATIO
    # *************************************************************************
    # df_["pkg_gfa"] = 0
    # # seek parking GFA and add up
    # cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
    #     "LargestPropertyUseType"]
    # for c in cols:
    #     gfa = c + "GFA"
    #     is_pkg = df_[c].str.lower().str.contains(r'parking', na=False)
    #     df_[gfa].where(is_pkg, 0, inplace=True)
    #     # add GFA to total
    #     df_["pkg_gfa"] += df_[gfa]
    # # keep highest GFA
    # df_.loc[df_["pkg_gfa"] > df_["PropertyGFAParking"],
    #     "PropertyGFAParking"] = df_["pkg_gfa"]
    # # apply % on total GFA
    # df_["ParkingRatio"] = df_["PropertyGFAParking"] / df_["PropertyGFATotal"]
    # df_.drop(["pkg_gfa"], axis=1, inplace=True)

    # NON-RESIDENTIAL RATIO (ALL USAGE FEATS COMPILATION)
    # *************************************************************************
    # df_["non_res_gfa"] = 0
    # lput_notna = df_["LargestPropertyUseType"] != np.NaN
    # cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
    #     "LargestPropertyUseType"
    # ]
    # # LargestPropertyUseType != NaN
    # for c in cols:
    #     gfa = c + "GFA"
    #     is_res = df_[c].str.lower().str.contains(
    #         r'(?<!(non))(residential|multifamily|residence)', na=True)
    #     df_[gfa].mask(lput_notna & is_res, 0, inplace=True)
    #     # add GFA to total
    #     df_["non_res_gfa"] += df_[gfa]
    # # LargestPropertyUseType == NaN
    # zero_non_res_gfa = df_["non_res_gfa"] != 0
    # is_res = df_["PrimaryPropertyType"].str.lower().str.contains(
    #     r'(?<!(non))(residential|multifamily|residence)', na=True)
    # df_["non_res_gfa"].where(lput_notna & is_res | zero_non_res_gfa,
    #     df_["PropertyGFATotal"], inplace=True)
    # # apply % on total GFA
    # df_["NonResidentialRatio"] = df_["non_res_gfa"] / df_["PropertyGFATotal"]
    # df_.loc[df_["NonResidentialRatio"] > 1, "NonResidentialRatio"] = 1
    # df_.drop(["non_res_gfa"], axis=1, inplace=True)

    # DROP POPULATION UNDER A NON-RESIDENTIAL RATIO
    # *************************************************************************
    # df_ = df_.loc[(df_["NonResidentialRatio"] >= min_nr_pct)]

    # CHANGE RAW ENERGY VALUES TO ENERGY INTENSITY
    # *************************************************************************
    # df_["SteamUse_I(kBtu/sf)"] = df_["SteamUse(kBtu)"]\
    #     / df_["PropertyGFATotal"]
    # df_["Electricity_I(kBtu/sf)"] = df_["Electricity(kBtu)"]\
    #     / df_["PropertyGFATotal"]
    df_["NaturalGas_I(kBtu/sf)"] = df_["NaturalGas(kBtu)"]\
        / df_["PropertyGFATotal"]
        
    # SET MINIMUM NUMBER OF FLOORS TO 1 AND ADD AREA PER FLOOR FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofFloors"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofFloors"] = 1
    df_["AreaPerFloor(sf)"] = df_["NumberofFloors"]\
        / df_["PropertyGFATotal"]

    # SET MINIMUM NUMBER OF BUILDINGS TO 1 AND ADD AREA PER BUILDING FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofBuildings"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofBuildings"] = 1
    df_["AreaPerBldg(sf)"] = df_["NumberofBuildings"]\
        / df_["PropertyGFATotal"]

    # CHASE STATISTICAL OUTLIERS
    # *************************************************************************
    df_ = df_.loc[df_["GHGEmissionsIntensity"] <= 20] # 2 individuals
    df_ = df_.loc[df_["AreaPerFloor(sf)"] <= 0.004] # 1 individual
    # df_ = df_.loc[df_["BuildingRatio"] >= 0.4] # 19 individuals
    df_ = df_.loc[df_["NumberofBuildings"] <= 20] # 3 individuals
    df_ = df_.loc[df_["NumberofFloors"] <= 40] # 16 individuals
    # df_ = df_.loc[df_["ParkingRatio"] <= 0.8] # 8 individuals
    df_ = df_.loc[df_["PropertyGFATotal"] <= 2000000] # 1 individual
    # df_ = df_.loc[df_["SteamUse_I(kBtu/sf)"] <= 100] # 6 individuals
    # df_ = df_.loc[df_["Electricity_I(kBtu/sf)"] <= 350] # 6 individuals

    # DELETE LAST USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "BuildingType",
        "PrimaryPropertyType",
        "LargestPropertyUseType",
        "LargestPropertyUseTypeGFA",
        "SecondLargestPropertyUseType",
        "SecondLargestPropertyUseTypeGFA",
        "ThirdLargestPropertyUseType",
        "ThirdLargestPropertyUseTypeGFA",
        "PropertyGFABuilding(s)", # => BuildingRatio
        # "PropertyGFAParking", # => ParkingRatio
        # "SteamUse(kBtu)",
        # "Electricity(kBtu)",
        "NaturalGas(kBtu)",
        "NumberofFloors",
        "NumberofBuildings",
    ], axis=1, inplace=True)

    # DROP NANS EXCEPT FOR ENERGY STAR SCORE FEATURE
    # *************************************************************************
    feats = ["ENERGYSTARScore"]
    df_.dropna(subset=df_.columns.difference(feats), inplace=True)

    # SORT COLUMNS
    # *************************************************************************
    df_.sort_index(axis=1, inplace=True)

    # SHOW GLOBAL IMPACT
    if verbose:
        logging.info("""\n***************************************************
    👇👇   GLOBAL IMPACT  👇👇""")
        impact(df, df_)

    return df_

def X_y_splitter(df, target, random_state=42, valid=False, verbose=False):
    """
    Splits a dataframe in random train, test and even validation samples,
    plus same for the target.

    Inputs:
    • df: original dataframe
    • target: targetted feature (string)
    • random_state: for randomization fixing (int)
    • valid: if a validation split is needed (bool, default = False)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 4 or 6 random dataframe samples

    Requirements: pandas, sklearn, logging
    """

    X = df.copy()
    X.drop(target, axis=1, inplace=True)
    y = df[target]

    if verbose:
        logging.info(f"{X.shape = }, {y.shape = }")


    if valid:
        # train / test : 70-30%
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=random_state)

        # validation / test : 50-50% (= 15-15% of total data)
        X_val, X_test, y_val, y_test = train_test_split(
            X_test, y_test, test_size=0.5, random_state=random_state)

        if verbose:
            logging.info(f"{X_train.shape = }, {y_train.shape = }\n" +
                f"{X_val.shape = }, {y_val.shape = }\n" +
                f"{X_test.shape = }, {y_test.shape = }")

        return X_train, X_val, X_test, y_train, y_val, y_test

    # train / test : 80-20%
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state)

    if verbose:
        logging.info(f"{X_train.shape = }, {y_train.shape = }\n" +
            f"{X_test.shape = }, {y_test.shape = }")

    return X_train, X_test, y_train, y_test

def X_splitter(df, random_state=42, valid=False, verbose=False):
    """
    Splits a dataframe in random train, test and even validation samples.

    Inputs:
    • df: original dataframe
    • random_state: for randomization fixing (int)
    • valid: if a validation split is needed (bool, default = False)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 2 or 3 random dataframe samples

    Requirements: pandas, sklearn, logging
    """

    if valid:
        # train / test : 70-30%
        X_train, X_test = train_test_split(
            df, test_size=0.3, random_state=random_state)

        # validation / test : 50-50% (= 15-15% of total data)
        X_val, X_test = train_test_split(
            X_test, test_size=0.5, random_state=random_state)

        if verbose:
            logging.info(f"{df.shape = }, {X_train.shape = }\n" +
                f"{X_val.shape = }, {X_test.shape = }")

        return X_train, X_val, X_test

    # train / test : 80-20%
    X_train, X_test = train_test_split(
        df, test_size=0.2, random_state=random_state)

    if verbose:
        logging.info(f"{df.shape = }, {X_train.shape = }, {X_test.shape = }")

    return X_train, X_test

def y_splitter(df, target, verbose=False):
    """
    Separates a dataframe from its target.

    Inputs:
    • df: original dataframe
    • target: targetted feature (string)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: 1 dataframe and 1 series (target)

    Requirements: pandas, sklearn, logging
    """

    X = df.copy()
    X.drop(target, axis=1, inplace=True)
    y = df[target]

    if verbose:
        logging.info(f"{X.shape = }, {y.shape = }")

    return X, y

def compare_df(y_train, y_test, name):
    """
    Compare quickly 2 target Series splits.

    Input: 2 dataframes or series ("train" and "test")
    Output: comparison dataframe

    Requirements: numpy, pandas
    """

    compere = pd.DataFrame(columns=["pop", "min", "max", "mean", "med", "std"])
    
    compere.loc["y_train_" + str(name)] = [y_train.shape[0], y_train.min(),
        y_train.max(), y_train.mean(), y_train.median(), y_train.std()]
    
    compere.loc["y_test_" + str(name)] = [y_test.shape[0], y_test.min(),
        y_test.max(), y_test.mean(), y_test.median(), y_test.std()]
    
    return compere

def process_all(df, non_res_min_usage, random_state, target, scores_df,
    verbose=False):
    """
    All-in-one processing, for parameters comparisons.

    scores_df = pd.DataFrame(columns=["rdm_st", "metric", "LinReg", "SVR",
        "RdmForestReg", "GradBoostReg", "MEAN", "MEDIAN"])
    """

    df_ = df.copy()

    # PARAMS
    # *************************************************************************
    non_res_min_usage = non_res_min_usage
    random_state = random_state
    target = target
    classif_cols = ["Neighborhood"]

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)

    # REMAINING PARAMS (must be done after cleaning)
    # *************************************************************************
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PIPELINES DEFINITIONS
    # *************************************************************************
    col_transf = ColumnTransformer(
        [
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
    )

    imputer = Pipeline(
        [
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    preprocessor = Pipeline(
        [
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    # SPLIT
    # *************************************************************************
    X_train, X_test, y_train, y_test = X_y_splitter(
        df_, target, random_state)
    
    # -> to dataframe
    X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
            columns=preprocessor[0:].get_feature_names_out())
    X_test = pd.DataFrame(preprocessor.fit_transform(X_test),
            columns=preprocessor[0:].get_feature_names_out())
    
    # for further comparison
    y_split = compare_df(y_train, y_test, random_state)

    # ESTIMATORS
    # *************************************************************************
    models_names = ["Dummy", "LinReg", "SVR", "RdmForestReg", "GradBoostReg"]
    
    # pipelines
    pipelines = [
        Pipeline([('Dummy', DummyRegressor(strategy="mean"))]),
        Pipeline([('LinReg', LinearRegression())]),
        Pipeline([('SVR', svm.SVR(kernel='linear'))]),
        Pipeline([('RdmForestReg', RandomForestRegressor())]),
        Pipeline([('GradBoostReg', GradientBoostingRegressor())]),
    ]

    # scores (DF preparation)
    row_name = "rdm_st_" + str(random_state)
    scores_df.loc[row_name + "_r2"] = "-"
    scores_df.loc[row_name + "_mae"] = "-"
    scores_df.loc[row_name + "_rmse"] = "-"
    scores_df.loc[row_name + "_ttime"] = "-"

    scores_df["rdm_st"][row_name + "_r2"] = random_state
    scores_df["metric"][row_name + "_r2"] = "r2"

    scores_df["rdm_st"][row_name + "_mae"] = random_state
    scores_df["metric"][row_name + "_mae"] = "mae"

    scores_df["rdm_st"][row_name + "_rmse"] = random_state
    scores_df["metric"][row_name + "_rmse"] = "rmse"
    
    scores_df["rdm_st"][row_name + "_ttime"] = random_state
    scores_df["metric"][row_name + "_ttime"] = "seconds"

    # loop over estimators
    for p, name in zip(pipelines, models_names):
        start = time.time()
        p.fit(X_train, y_train)
        end = time.time()
        fit_time = end - start

        r2 = round(r2_score(y_test, p.predict(X_test)), 3)
        mae = round(mean_absolute_error(y_test, p.predict(X_test)), 3)
        rmse = round(mean_squared_error(y_test, p.predict(X_test),
            squared = False), 3)

        # save scores in DF
        if name != "Dummy":
            scores_df[name][row_name + "_r2"] = r2
            scores_df[name][row_name + "_mae"] = mae
            scores_df[name][row_name + "_rmse"] = rmse
            scores_df[name][row_name + "_ttime"] = fit_time

    # means and medians for estimators
    scores_df["MEAN"][row_name + "_r2"] = round(
        scores_df.loc[row_name + "_r2"][2:6].mean(), 3)
    scores_df["MEAN"][row_name + "_mae"] = round(
        scores_df.loc[row_name + "_mae"][2:6].mean(), 3)
    scores_df["MEAN"][row_name + "_rmse"] = round(
        scores_df.loc[row_name + "_rmse"][2:6].mean(), 3)

    scores_df["MEDIAN"][row_name + "_r2"] = round(
        scores_df.loc[row_name + "_r2"][2:6].median(), 3)
    scores_df["MEDIAN"][row_name + "_mae"] = round(
        scores_df.loc[row_name + "_mae"][2:6].median(), 3)
    scores_df["MEDIAN"][row_name + "_rmse"] = round(
        scores_df.loc[row_name + "_rmse"][2:6].median(), 3)

    if verbose:
        display(y_split)
        display(scores_df)

    return y_split, scores_df

def display_r2(action):
    """
    """
    
    print(action[0])

    scores = action[2]

    # r2 results
    r2_all = scores.loc[scores["metric"] == "r2"]
    # global mean score for a same random state
    r2_all.loc["TOTAL"] = [
        "-",
        "-",
        round(r2_all["LinReg"].mean(), 3),
        round(r2_all["SVR"].mean(), 3),
        round(r2_all["RdmForestReg"].mean(), 3),
        round(r2_all["GradBoostReg"].mean(), 3),
        round(r2_all["MEAN"].mean(), 3),
        round(r2_all["MEDIAN"].mean(), 3),
    ]
    display(r2_all)

    # graphic figure
    df_ = r2_all.drop("TOTAL", axis=0)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["LinReg"],
        mode='lines+markers', name='LinReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["SVR"],
        mode='lines+markers', name='SVR'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["RdmForestReg"],
        mode='lines+markers', name='RdmForestReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["GradBoostReg"],
        mode='lines+markers', name='GradBoostReg'))
    fig.add_trace(go.Scatter(x=df_["rdm_st"], y=df_["MEAN"],
        mode='lines+markers', name='MEAN'))
    fig.show()

    return

def display_all_r2(scores_list):
    """
    Displays a plot of all r2 scores for step by step analysis.
    """

    columns = ['LinReg', 'SVR', 'RdmForestReg', 'GradBoostReg', "mean",
        "median"]
    full_r2 = pd.DataFrame(columns=columns)

    for f in scores_list:
        # get variable name
        name = f[0]
        
        _ = f[3].loc[f[3]["metric"] == "r2"]
        linreg = round(_["LinReg"].mean(), 3)
        svr = round(_["SVR"].mean(), 3)
        rdmforest = round(_["RdmForestReg"].mean(), 3)
        gradboost = round(_["GradBoostReg"].mean(), 3)

        full_r2.loc[name] = [
            linreg,
            svr,
            rdmforest,
            gradboost,
            round(np.mean([linreg, svr, rdmforest, gradboost]), 3),
            round(np.median([linreg, svr, rdmforest, gradboost]), 3),
        ]

        # mask if too high / low
        full_r2.mask(full_r2 > 3, 3, inplace=True)
        full_r2.mask(full_r2 < -3, -3, inplace=True)
 
    # graphic figure
    layout = go.Layout(yaxis=dict(range=[-0.5, 1]))
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["LinReg"],
        mode='lines+markers', name='LinReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["SVR"],
        mode='lines+markers', name='SVR'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["RdmForestReg"],
        mode='lines+markers', name='RdmForestReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["GradBoostReg"],
        mode='lines+markers', name='GradBoostReg'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["mean"],
        mode='lines+markers', name='mean'))
    fig.add_trace(go.Scatter(x=full_r2.index, y=full_r2["median"],
        mode='lines+markers', name='median'))
    fig.show()

    print("""save_actions = BASE: drop duplicates (0), keep compliants only, clean neighborhood, drop useless feats, drop nans except ENERGYSTARScore, sort columns
save_actions_2    = BASE + filter GHGEmissionsIntensity, NumberofBuildings, NumberofFloors, PropertyGFATotal (22)
save_actions_3    = ........ + create BuildingRatio feature
save_actions_4    = ............ + drop PropertyGFABuilding(s) feature
save_actions_5    = ................ + filter BuildingRatio (19)
save_actions_6    = .................|.. + create ParkingRatio feature
save_actions_6_1  = .................|...... + drop PropertyGFAParking feature
save_actions_6_2  = .................|.......... + filter ParkingRatio (8)
save_actions_7    = ................ + create AreaPerFloor(sf) and AreaPerBldg(sf) features
save_actions_7_1  = .................... + drop NumberofFloors and NumberofBuildings features
save_actions_7_2  = ........................ + filter AreaPerFloor(sf) (1)
save_actions_8    = ............................ + create SteamUse_I(kBtu/sf), Electricity_I(kBtu/sf) and NaturalGas_I(kBtu/sf) features
save_actions_8_1  = .............................|.. + drop SteamUse(kBtu), Electricity(kBtu) and NaturalGas(kBtu) features
save_actions_8_2  = .............................|...... + filter SteamUse_I(kBtu/sf) (6), Electricity_I(kBtu/sf) (6)
save_actions_9    = .............................|.......... + create NonResidentialRatio feature
save_actions_9_1  = .............................|.............. + filter NonResidentialRatio >= min_nr_pct (often > 1000)
save_actions_10   = .............................|.................. + create ParkingRatio, drop PropertyGFAParking, filter ParkingRatio (8)
save_actions_11   = ............................ + create NonResidentialRatio + filter >= min_nr_pct (often > 1000)
save_actions_11_1 = .............................|.. + create ParkingRatio, drop PropertyGFAParking feature, filter ParkingRatio (8)
save_actions_12   = ............................ + STEAMUSE: create SteamUse_I(kBtu/sf), drop SteamUse(kBtu), filter SteamUse_I(kBtu/sf) (6)
save_actions_13   = ............................ + ELECTRICITY: create Electricity_I(kBtu/sf), drop Electricity(kBtu), filter Electricity_I(kBtu/sf) (6)
save_actions_14   = ............................ + NATURALGAS: create NaturalGas_I(kBtu/sf), drop NaturalGas(kBtu)
""")

    return


# <a id='toc1_'></a>[Estimateurs : comparaison et recherche de fuite](#toc0_)

In [4]:
%%time

non_res_min_usage = 0.0001
target = "GHGEmissionsIntensity"
splits = pd.DataFrame()
scores = pd.DataFrame(columns=["rdm_st", "metric", "LinReg", "SVR",
    "RdmForestReg", "GradBoostReg", "MEAN", "MEDIAN"])

random_state = range(10)

for rs in random_state:
    y_split, _ = process_all(data_raw, non_res_min_usage, rs, target,
        scores, verbose=False)
    splits = pd.concat([splits, y_split])

CPU times: total: 1min 16s
Wall time: 1min 7s


# <a id='toc2_'></a>[Tableau comparatif](#toc0_)

In [5]:
txt = """ACTIONS:
- drop duplicates
- keep compliants only
- clean neighborhood
- drop useless feats

PLUS
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature

- create `AreaPerFloor(sf)` feature
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature  
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

- create `NaturalGas_I(kBtu/sf)` feature
- drop `NaturalGas(kBtu)` feature

- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)

AND
- drop last useless feats
- drop nans except ENERGYSTARScore
- sort columns
"""

# save_actions_14 = [txt, splits, scores]
# %store save_actions_14

# <a id='toc3_'></a>[Méthode pas à pas](#toc0_)

Application des divers traitements et analyse pas à pas des résultats pour détecter la fuite de données éventuelle.

## Traitement de base

(résultats dans variable `save_actions`)

BASIC ACTIONS:
- drop duplicates
- keep compliants only
- clean neighborhood
- drop useless feats

AND
- drop last useless feats
- drop nans except ENERGYSTARScore
- sort columns

**➡️ -21395404395282247680.0**

mais si on enlève la régression linéaire :  
**➡️ -0.674**

In [1]:
%store -r save_actions
# display_r2(save_actions)

## save_actions_2

BASE +
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)

**➡️ -23870982611243671552.0**

mais si on enlève la régression linéaire :  
**➡️ 0.446**

In [7]:
%store -r save_actions_2
# display_r2(save_actions_2)

## save_actions_3

BASE +  
save_actions_2  
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)

\+ create `BuildingRatio` feature

**➡️ -64882775773597270016.0**

mais si on enlève la régression linéaire :  
**➡️ 0.443**

In [8]:
%store -r save_actions_3
# display_r2(save_actions_3)

## save_actions_4

BASE +  
save_actions_3
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature

\+ drop `PropertyGFABuilding(s)` feature

**➡️ 0.361**

✅ règle le souci de régression linéaire

In [9]:
%store -r save_actions_4
# display_r2(save_actions_4)

## save_actions_5

BASE +  
save_actions_4
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature

\+ filter `BuildingRatio` >= 0.4 (19 individuals)

**➡️ 0.078**

❌ chute modèle : abandon

In [10]:
%store -r save_actions_5
# display_r2(save_actions_5)

## save_actions_6

BASE +  
save_actions_4
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature

\+ create `ParkingRatio` feature

**➡️ 0.1**

❌ chute modèle → approfondissement

In [11]:
%store -r save_actions_6
# display_r2(save_actions_6)

## save_actions_6.1

BASE +  
save_actions_6
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `ParkingRatio` feature

\+ drop `PropertyGFAParking` feature

**➡️ 0.096**

❌ toujours bof

In [12]:
%store -r save_actions_6_1
# display_r2(save_actions_6_1)

## save_actions_6.2

BASE +  
save_actions_6.1
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `ParkingRatio` feature
- drop `PropertyGFAParking` feature

\+ filter `ParkingRatio` <= 0.8 (8 individuals)

**➡️ -0.092**

❌ encore chuté : abandon de la branche

In [13]:
%store -r save_actions_6_2
# display_r2(save_actions_6_2)

## save_actions_7

BASE +  
save_actions_4
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature

\+ create `AreaPerFloor(sf)` feature  
\+ create `AreaPerBldg(sf)` feature

**➡️ 0.31**

↘ un peu baissé

In [14]:
%store -r save_actions_7
# display_r2(save_actions_7)

## save_actions_7_1

BASE +  
save_actions_7
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature

\+ drop `NumberofFloors` feature  
\+ drop `NumberofBuildings` feature

**➡️ 0.327**

↗️ légère hausse du r2

In [15]:
%store -r save_actions_7_1
# display_r2(save_actions_7_1)

## save_actions_7_2

BASE +  
save_actions_7_1
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature

\+ filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

**➡️ 0.328**

≈ avant

In [16]:
%store -r save_actions_7_2
# display_r2(save_actions_7_2)

## save_actions_8

BASE +  
save_actions_7_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

\+ create `SteamUse_I(kBtu/sf)` feature  
\+ create `Electricity_I(kBtu/sf)` feature  
\+ create `NaturalGas_I(kBtu/sf)` feature

**➡️ 0.603**

↗↗ belle augmentation

In [17]:
%store -r save_actions_8
# display_r2(save_actions_8)

## save_actions_8_1

BASE +  
save_actions_8
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `SteamUse_I(kBtu/sf)` feature
- create `Electricity_I(kBtu/sf)` feature
- create `NaturalGas_I(kBtu/sf)` feature

\+ drop `SteamUse(kBtu)` feature  
\+ drop `Electricity(kBtu)` feature  
\+ drop `NaturalGas(kBtu)` feature  

**➡️ 0.603**

== kif-kif

In [18]:
%store -r save_actions_8_1
# display_r2(save_actions_8_1)

## save_actions_8_2

BASE +  
save_actions_8_1
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `SteamUse_I(kBtu/sf)` feature
- create `Electricity_I(kBtu/sf)` feature
- create `NaturalGas_I(kBtu/sf)` feature
- drop `SteamUse(kBtu)` feature
- drop `Electricity(kBtu)` feature
- drop `NaturalGas(kBtu)` feature

\+ filter `SteamUse_I(kBtu/sf)` <= 100 (6 individuals)  
\+ filter `Electricity_I(kBtu/sf)` <= 350 (6 individuals)

**➡️ 0.862**

❓❓❓❓ très bien ou fuite ?

In [19]:
%store -r save_actions_8_2
# display_r2(save_actions_8_2)

## save_actions_9

BASE +  
save_actions_8_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `SteamUse_I(kBtu/sf)` feature
- create `Electricity_I(kBtu/sf)` feature
- create `NaturalGas_I(kBtu/sf)` feature
- drop `SteamUse(kBtu)` feature
- drop `Electricity(kBtu)` feature
- drop `NaturalGas(kBtu)` feature
- filter `SteamUse_I(kBtu/sf)` <= 100 (6 individuals)  
- filter `Electricity_I(kBtu/sf)` <= 350 (6 individuals)

\+ create `NonResidentialRatio` feature

**➡️ 0.863**

== idem

In [20]:
%store -r save_actions_9
# display_r2(save_actions_9)

## save_actions_9_1

BASE +  
save_actions_9
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `SteamUse_I(kBtu/sf)` feature
- create `Electricity_I(kBtu/sf)` feature
- create `NaturalGas_I(kBtu/sf)` feature
- drop `SteamUse(kBtu)` feature
- drop `Electricity(kBtu)` feature
- drop `NaturalGas(kBtu)` feature
- filter `SteamUse_I(kBtu/sf)` <= 100 (6 individuals)  
- filter `Electricity_I(kBtu/sf)` <= 350 (6 individuals)
- create `NonResidentialRatio` feature

\+ filter `NonResidentialRatio` >= min_nr_pct (often > 1000 individuals)

**➡️ 0.934**

❓❓❓❓ idem : fuite ou bien ?

In [21]:
%store -r save_actions_9_1
# display_r2(save_actions_9_1)

## save_actions_10

BASE +  
save_actions_9_1
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `SteamUse_I(kBtu/sf)` feature
- create `Electricity_I(kBtu/sf)` feature
- create `NaturalGas_I(kBtu/sf)` feature
- drop `SteamUse(kBtu)` feature
- drop `Electricity(kBtu)` feature
- drop `NaturalGas(kBtu)` feature
- filter `SteamUse_I(kBtu/sf)` <= 100 (6 individuals)  
- filter `Electricity_I(kBtu/sf)` <= 350 (6 individuals)
- create `NonResidentialRatio` feature
- filter `NonResidentialRatio` >= min_nr_pct (often > 1000 individuals)

\+ create `ParkingRatio` feature  
\+ drop `PropertyGFAParking` feature  
\+ filter `ParkingRatio` <= 0.8 (8 individuals)

**➡️ 0.961**

↗️ cette fois, légère amélioration

In [22]:
%store -r save_actions_10
# display_r2(save_actions_10)

## save_actions_11

BASE +  
save_actions_7_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

\+ create `NonResidentialRatio` feature  
\+ filter `NonResidentialRatio` >= min_nr_pct (often > 1000 individuals)

**➡️ 0.007**

❌ chute !

In [23]:
%store -r save_actions_11
# display_r2(save_actions_11)

## save_actions_11_1

BASE +  
save_actions_11
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)
- create `NonResidentialRatio` feature  
- filter `NonResidentialRatio` >= min_nr_pct (often > 1000 individuals)

\+ create `ParkingRatio` feature  
\+ drop `PropertyGFAParking` feature  
\+ filter `ParkingRatio` <= 0.8 (8 individuals)

**➡️ 0.212**

↗️ mieux mais toujours pas top : fuite pas ici

In [24]:
%store -r save_actions_11_1
# display_r2(save_actions_11_1)

## save_actions_12

BASE +  
save_actions_7_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

**TEST STEAM_USE**  
\+ create `SteamUse_I(kBtu/sf)` feature  
\+ drop `SteamUse(kBtu)` feature  
\+ filter `SteamUse_I(kBtu/sf)` <= 100 (6 individuals)  

**➡️ 0.336**

≈ kif-kif

In [25]:
%store -r save_actions_12
# display_r2(save_actions_12)

## save_actions_13

BASE +  
save_actions_7_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

**TEST ELECTRICITY**  
\+ create `Electricity_I(kBtu/sf)` feature  
\+ drop `Electricity(kBtu)` feature  
\+ filter `Electricity_I(kBtu/sf)` <= 350 (6 individuals)

**➡️ 0.579**

↗ belle augmentation (0.328 au test 7_2)

In [26]:
%store -r save_actions_13
# display_r2(save_actions_13)

## save_actions_14

BASE +  
save_actions_7_2
- filter `GHGEmissionsIntensity` <= 20 (2 individuals)
- filter `NumberofBuildings` <= 20 (3 individuals)
- filter `NumberofFloors` <= 40 (16 individuals)
- filter `PropertyGFATotal` <= 2000000 (1 individual)
- create `BuildingRatio` feature
- drop `PropertyGFABuilding(s)` feature
- create `AreaPerFloor(sf)` feature  
- create `AreaPerBldg(sf)` feature
- drop `NumberofFloors` feature
- drop `NumberofBuildings` feature
- filter `AreaPerFloor(sf)` <= 0.004 (1 individual)

**TEST NATURALGAS**  
\+ create `NaturalGas_I(kBtu/sf)` feature  
\+ drop `NaturalGas(kBtu)` feature

**➡️ 0.614**

↗ belle augmentation (0.328 au test 7_2)

In [27]:
%store -r save_actions_14
# display_r2(save_actions_14)

# <a id='toc4_'></a>[Enregistrement des traitements](#toc0_)

Enregistrement des traitements dans une liste et export Pickle :

In [28]:
%%script echo "for Pickle saving, just exec once"

scores_list = [save_actions, save_actions_2, save_actions_3, save_actions_4,
    save_actions_5, save_actions_6, save_actions_6_1, save_actions_6_2,
    save_actions_7, save_actions_7_1, save_actions_7_2, save_actions_8,
    save_actions_8_1, save_actions_8_2, save_actions_9, save_actions_9_1,
    save_actions_10, save_actions_11, save_actions_11_1, save_actions_12,
    save_actions_13, save_actions_14]

score_names = ["save_actions", "save_actions_2", "save_actions_3",
    "save_actions_4", "save_actions_5", "save_actions_6", "save_actions_6_1",
    "save_actions_6_2", "save_actions_7", "save_actions_7_1",
    "save_actions_7_2", "save_actions_8", "save_actions_8_1",
    "save_actions_8_2", "save_actions_9", "save_actions_9_1",
    "save_actions_10", "save_actions_11", "save_actions_11_1",
    "save_actions_12", "save_actions_13", "save_actions_14"]

for i, e in enumerate(scores_list):
    e.insert(0, score_names[i])

pickle.dump(scores_list, open("data_leak_scores.p", "wb"))

Couldn't find program: 'echo'


# <a id='toc5_'></a>[Scores (r2) selon les traitements](#toc0_)

In [29]:
scores_list = pickle.load(open("data_leak_scores.p", "rb"))
display_all_r2(scores_list)

save_actions = BASE: drop duplicates (0), keep compliants only, clean neighborhood, drop useless feats, drop nans except ENERGYSTARScore, sort columns
save_actions_2    = BASE + filter GHGEmissionsIntensity, NumberofBuildings, NumberofFloors, PropertyGFATotal (22)
save_actions_3    = ........ + create BuildingRatio feature
save_actions_4    = ............ + drop PropertyGFABuilding(s) feature
save_actions_5    = ................ + filter BuildingRatio (19)
save_actions_6    = .................|.. + create ParkingRatio feature
save_actions_6_1  = .................|...... + drop PropertyGFAParking feature
save_actions_6_2  = .................|.......... + filter ParkingRatio (8)
save_actions_7    = ................ + create AreaPerFloor(sf) and AreaPerBldg(sf) features
save_actions_7_1  = .................... + drop NumberofFloors and NumberofBuildings features
save_actions_7_2  = ........................ + filter AreaPerFloor(sf) (1)
save_actions_8    = ............................ + cr